In [2]:
import time
import numpy as np
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [74]:
opt = webdriver.ChromeOptions()
opt.add_argument("--incognito")
opt.add_argument("disable-infobars")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-gpu")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument("--headless=new")
driver = webdriver.Chrome(options=opt)

url = "https://www.target.com/s?searchTerm=christmas+decorations&category=0%7CAll%7Cmatchallpartial%7Call+categories&tref=typeahead%7Cterm%7C0%7Cchristmas+decorations%7Cchristmas+decorations%7C%7C%7Cservice%7C%7C%7C%7C%7Cfacets_sb&searchTermRaw=christmas+de"
driver.get(url)

wait = WebDriverWait(driver, 20)

In [6]:
wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@data-component-id="WEB-c_web_productgrid_v01"]')))

<selenium.webdriver.remote.webelement.WebElement (session="89f73f2a98085031edea78a194c1dd56", element="9BFE86D3C8F7456744E6DB1867EF88CF_element_37")>

In [15]:
wait.until(EC.visibility_of_element_located((By.XPATH, '//button[@id="select-custom-button-id"]')))
page_location = driver.find_element(By.XPATH, '//button[@id="select-custom-button-id"]')
page_amount = int(page_location.text.split()[-1])

In [17]:
links = []
total = 0
for i in range(page_amount):
    wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="styles__ProductListGridFadedLoading-sc-u8zdb1-0"]')))
    y = 1000
    for j in range(0,10):
        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += 500  
        time.sleep(1)
    
    decorations_location = driver.find_element(By.XPATH, '//div[@class="styles__ProductListGridFadedLoading-sc-u8zdb1-0"]')
    item_location = decorations_location.find_elements(By.XPATH, 'section/div/div')
    items = [i.find_element(By.XPATH, 'div/div/div[1]/div[1]/h3/div/div/a').get_attribute("href") for i in item_location if ("Sponsored" not in i.text)]

    total += len(items)
    links += items
      
    print(i)
    if i != page_amount-1:
        button = driver.find_element(By.XPATH, '//div[@class="styles__ButtonWithArrowWrapper-sc-l17a0m-4 cRjaPg"][2]')
        hover = ActionChains(driver).move_to_element(button)
        hover.perform()
        button.click()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [18]:
len(links)

1200

In [19]:
total

1200

In [20]:
len(links) != len(set(links))

True

In [21]:
len(set(links))

1106

In [22]:
links_nd = []
len([links_nd.append(i) for i in links if i not in links_nd])

1106

In [23]:
len(links_nd) != len(set(links))

False

In [25]:
driver.get(links_nd[11])

In [26]:
page = driver.page_source
soup = BeautifulSoup(page, "lxml")

In [27]:
cart = soup.find(class_="styles__StyledAddToCartColumn-sc-2vujr8-5 hrsYls")

In [28]:
print(cart.prettify())

<div class="styles__StyledAddToCartColumn-sc-2vujr8-5 hrsYls">
 <div class="styles__StyledAddToCartContainer-sc-y0ahq4-0 jhewfn">
  <div id="above-the-fold-information" tabindex="-1">
   <div class="h-margin-b-tiny">
    <ul class="styles__StyledProductDetailTitleRelatedLinks-sc-2vujr8-0 VDWUB">
     <li>
      <a class="styles__StyledLink-sc-vpsldm-0 eqjfDm" data-test="shopAllBrandLink" href="/b/northlight/-/N-4f1bz">
       <span>
        Shop all
        <!-- -->
        Northlight
       </span>
      </a>
     </li>
    </ul>
   </div>
   <div class="h-margin-b-tiny">
    <h1 class="styles__StyledHeading-sc-1xmf98v-0 cwBDOF h-text-bold h-margin-b-none" data-test="product-title" id="pdp-product-title-id">
     Northlight 13.5" Lighted Red Moose and Christmas Tree Wooden LED Battery Operated Decoration
    </h1>
   </div>
   <div class="h-margin-b-tiny h-display-flex">
    <div class="h-display-flex" data-test="ratingFeedbackContainer">
     <a aria-haspopup="true" class="styles__St

In [29]:
cart.find(id="pdp-product-title-id").text.strip()

'Northlight 13.5" Lighted Red Moose and Christmas Tree Wooden LED Battery Operated Decoration'

In [30]:
cart.find(class_="styles__PriceFullLineHeight-sc-1mh0sjm-0 dIfIJG style__StyledPriceFull-sc-1o3i6gc-1 OpJYy").text.strip()

'$35.99  reg $43.49Sale\xa0save $7.50 (17% off)When purchased online'

In [33]:
# Discount/Sale price
cart.find(class_="styles__CurrentPriceFontSize-sc-1mh0sjm-1 bksmYC").text

'$35.99'

In [34]:
oh = cart.find(class_="utils__ScreenReaderOnly-sc-1b93ups-0 hMtWwx").text.split()

In [38]:
oh[0]

'1'

In [39]:
oh[-2]

'1'

In [40]:
word = cart.find(class_="styles__StyledProductDetailTitleRelatedLinks-sc-2vujr8-0 VDWUB").text.split()[2:]

In [41]:
print(word)

['Northlight']


In [42]:
stopwords = ['Shop', 'all', 'View', 'similar', 'items']
resultwords  = [i for i in word if i not in stopwords]
result = " ".join(resultwords)
if result.endswith("View"):
    print(result[0:-4])

In [43]:
categories_loc = soup.find(class_="styles__StyledBreadcrumbsWrapper-sc-2vujr8-2 kwpxOw")#.text.split("/")

In [44]:
sub_categories_loc = categories_loc.find_all(class_="styles__StyledLink-sc-vpsldm-0 eqjfDm")

In [45]:
sub_categories_loc[-2].text

'Christmas Decorations'

In [46]:
sub_categories_loc[-1].text

'Indoor Christmas Decorations'

In [47]:
stopwords = ['Shop', 'all', 'View', 'similar', 'items']

In [75]:
name = []
actual_price = []
on_sale_price = []
percent_off = []
review_count = []
rating = []
category = []
sub_category = []
brand = []

for i in links_nd:
    driver.get(i)
    
    wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="styles__StyledAddToCartColumn-sc-2vujr8-5 hrsYls"]')))
    wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="styles__PriceFullLineHeight-sc-1mh0sjm-0 dIfIJG style__StyledPriceFull-sc-1o3i6gc-1 OpJYy"]')))
    
    page = driver.page_source
    soup = BeautifulSoup(page, "lxml")
    
    cart = soup.find(class_="styles__StyledAddToCartColumn-sc-2vujr8-5 hrsYls")
    
    name.append(cart.find(id="pdp-product-title-id").text.strip())
    
    price_loc = cart.find(class_="styles__PriceFullLineHeight-sc-1mh0sjm-0 dIfIJG style__StyledPriceFull-sc-1o3i6gc-1 OpJYy").text.strip()
    if "Sale" in price_loc:
        actual_price.append(cart.find(class_="h-text-line-through").text)
        on_sale_price.append(cart.find(class_="styles__CurrentPriceFontSize-sc-1mh0sjm-1 bksmYC").text)
        percent_off.append(cart.find(class_="h-text-grayDark h-text-normal h-text-sm").text.split()[2][1:])
    else:
        actual_price.append(cart.find(class_="styles__CurrentPriceFontSize-sc-1mh0sjm-1 bksmYC").text)
        on_sale_price.append("Not on Sale")
        percent_off.append("0%")
    
    review_info = cart.find(class_="utils__ScreenReaderOnly-sc-1b93ups-0 hMtWwx").text.split()
    rating.append(review_info[0])
    review_count.append(review_info[-2])
    
    brand_word = cart.find(class_="styles__StyledProductDetailTitleRelatedLinks-sc-2vujr8-0 VDWUB").text.split()
    resultwords  = [i for i in brand_word if i.lower() not in stopwords]
    result = " ".join(resultwords)
    if result.endswith("View"):
        brand.append(result[0:-4])
    else:
        brand.append(result)
    
    categories_loc = soup.find(class_="styles__StyledBreadcrumbsWrapper-sc-2vujr8-2 kwpxOw")
    sub_categories_loc = categories_loc.find_all(class_="styles__StyledLink-sc-vpsldm-0 eqjfDm")
    category.append(sub_categories_loc[-2].text)
    sub_category.append(sub_categories_loc[-1].text)

In [76]:
len(name)

1106

In [77]:
len(actual_price)

1106

In [78]:
len(on_sale_price)

1106

In [79]:
len(percent_off)

1106

In [80]:
len(review_count)

1106

In [81]:
len(rating)

1106

In [82]:
len(category)

1106

In [83]:
len(sub_category)

1106

In [84]:
len(brand)

1106

In [85]:
brand

['Shop Wondershop',
 'Shop Wondershop',
 'Shop Mr. Christmas',
 'Shop Northlight',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Northlight',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Wondershop',
 'Shop Northlight',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Mr. Christmas',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Mr. Christmas',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Mr. Christmas',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop Northlight',
 'Shop

In [86]:
brand = [i[5:] for i in brand]

In [87]:
data = {"Name": name, "Actual Price": actual_price, "Sale Price": on_sale_price, "Discount %": percent_off, "Rating": rating, "Number of Reviews": review_count, "Category": category, "Sub-Category": sub_category, "Brand": brand}

In [88]:
#links_nd[3]

In [89]:
df = pd.DataFrame.from_dict(data)
df.head()

,Name,Actual Price,Sale Price,Discount %,Rating,Number of Reviews,Category,Sub-Category,Brand
0,Santa Fabric Christmas Figurine with Blue Gift...,$4.25,Not on Sale,0%,4.2,13,Christmas Decorations,Indoor Christmas Decorations,Wondershop
1,"6.75"" Featherly Friends Bird Dressed as Elf Fa...",$4.25,Not on Sale,0%,4.8,44,Christmas Decorations,Indoor Christmas Decorations,Wondershop
2,"Mr. Christmas 17"" Tabletop Super Climber Anima...",$49.99,$19.99,60%,3.9,29,Christmas Decorations,Indoor Christmas Decorations,Mr. Christmas
3,"Northlight 7"" LED Lighted Magnolia Leaf, Pinec...",$43.49,$35.99,17%,4,2,Christmas Decorations,Indoor Christmas Decorations,Northlight
4,Mr. Christmas Animated Mini Carnival Music Box...,$34.99,$19.99,43%,3.6,11,Christmas Decorations,Indoor Christmas Decorations,Mr. Christmas


In [90]:
#df.to_csv("Datasets/target_christmas_deco.csv")

In [91]:
df.to_csv("Datasets/target_christmas_deco_2.csv")